<center>
# Linear algebra in
<img width=400 src=julia.png>
## Andreas Noack
### September 2015
### Computer Science and Artificial Intelligence Laboratory<br>Massachusetts Institute of Technology
</center>
## Outline
 - What is Julia
  - The people behind
  - Two language problem
 - Introduction to linear algebra
  - Basic linear algebra
  - Factorizations
  - Special matrices
 - Generic linear algebra
 - Fast linear algebra
 - The future!

## What is Julia?
 
### Founded by
 - Alan Edelman
 - Jeff Besanzon
 - Viral B. Shah
 - Stefan Starpinski
<img width=700 src=kahan.jpg>

### Why?
> In short, because we are greedy.

> ...

> We want a language ..., with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled.

from the [blog post](http://julialang.org/blog/2012/02/why-we-created-julia/) acompanying the first public version of Julia in February 2012.

### Status
As of yesterday evening
 - 399 contributors to [JuliaLang/julia](https://github.com/JuliaLang/julia)
<img width=700 src=world1.pdf>
Picture by Jiahao Chen

688 packages registered at [pkg.julialang.org](http://pkg.julialang.org/)
<img width=700 src=packages.svg>
Picture by Iain Dunning

### License
- Most of Julia's source is licensed under the [MIT license](https://opensource.org/licenses/MIT)
- Allows for commecial use of Julia without restriction
- Binaries are GPL because of linked libraries (primarily sparse linear algebra)
- Build flag to avoid GPL licensed components

### Two language problem - Julia is an outlier
Average of running times vs. average of lines to implement micro benchmark for some popular languages
<img width=700 src=SpeedVsLines.png>
Picture by Simon Danisch

### How?

### Mathematical syntax
Simple syntax with extensive unicode support that allows code to look like math

In [1]:
# eigenvalues
A = randn(3,3) |> t -> t + t'
λ = eigmax(A)
@show det(A - λ*I)

det(A - λ * I) = 6.7656021995418656e-15

6.7656021995418656e-15

In [2]:
# Fibonacci
f(n) = n < 2 ? 1 : f(n-1) + f(n-2)
for i = 0:10
    println("f($i) = $(f(i))")
end

f(0) = 1
f(1) = 1
f(2) = 2
f(3) = 3
f(4) = 5
f(5) = 8
f(6) = 13
f(7) = 21
f(8) = 34
f(9) = 55
f(10) = 89


## Basic linear algebra
Syntax very similar to MATLAB but there are some important differences. Define a matrix of random normal variates

In [3]:
A = randn(3,3)

3x3 Array{Float64,2}:
 -0.0828106  1.26905    1.6852  
  0.11839    0.581428  -0.701548
  1.42867    1.06831   -0.144026

and a vector of ones

In [4]:
x = ones(3)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type `Array{Float64,2}` but $x$ has type `Array{Float64,1}`. Julia defines the aliases `Vector{Type} = Array{Type,1}` and `Matrix{Type} = Array{Type,2}`. The destinction between vectors and matrices typically causes some confusion when coming from a MATLAB background.

Many of the basic operations are the same as in MATLAB
### Multiplication

In [5]:
b = A*x

3-element Array{Float64,1}:
  2.87143   
 -0.00172941
  2.35295   

### Transposition
As in MATLAB `A'` is the conjugate transpose whereas `A.'` is just the transpose

In [7]:
Asym = A + A'

3x3 Array{Float64,2}:
 -0.165621  1.38744   3.11386 
  1.38744   1.16286   0.36676 
  3.11386   0.36676  -0.288051

### Transposed multiplication
Julia allows us to write this without *

In [8]:
Apd = A'A

3x3 Array{Float64,2}:
  2.06196   1.49     -0.428373
  1.49      3.08982   1.57683 
 -0.428373  1.57683   3.3528  

### Solving linear systems 
The problem $Ax=b$ for square $A$ is solved by the `\` function.

In [9]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

### Least squares
When A is tall the `\` function calculates the least squares solution

In [10]:
A[:,1:2]\b

2-element Array{Float64,1}:
 0.115132
 1.93704 

The `\` function also works for rank deficient least squares problems. In this case, the least squares solution is not unique and Julia returns the solution with smallest norm

In [11]:
[A[:,1] A[:,1]]\b

2-element Array{Float64,1}:
 0.757432
 0.757432

### Underdetermined systems
Minimum norm solution is returned

In [12]:
A[1:2,:]\b[1:2]

3-element Array{Float64,1}:
 -0.0132692
  1.07662  
  0.892507 

# Factorizations
The `\` function hides how the problem is actually solved. Depending on the dimensions of `A`, different methods are chosen to solve the problem. An intermediate step in the solution is to calculate a factorization of the matrix `A`. Basically, a factorization of `A` is a way of expressing `A` as a product of triangular, unitary and permutation matrices. Julia defines a `Factorization` abstract type and several composite subtypes for actually storing factorizations. A `Factorization` object should be thought of as a representation of the matrix `A`.

###LU
When $A$ is square, problem is solved by factorizing the matrix $A = PLU$ where $P$ is a permutation matrix, $L$ is lower triangular unit diagonal and $U$ is upper triangular. Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [13]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}}(3x3 Array{Float64,2}:
  1.42867    1.06831   -0.144026
 -0.0579636  1.33097    1.67685 
  0.0828678  0.370331  -1.3106  ,[3,3,3],0)

The different parts of the factorization can be extracted by special indexing

In [14]:
Alu[:P]

3x3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [15]:
Alu[:L]

3x3 Array{Float64,2}:
  1.0        0.0       0.0
 -0.0579636  1.0       0.0
  0.0828678  0.370331  1.0

In [16]:
Alu[:U]

3x3 Array{Float64,2}:
 1.42867  1.06831  -0.144026
 0.0      1.33097   1.67685 
 0.0      0.0      -1.3106  

We can therefore compute the solution of $Ax=b$ from the factorization

In [16]:
Alu[:U]\(Alu[:L]\(Alu[:P]'b))

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

However, more importantly the `LU` type allows dispatch and we can solve the system by

In [17]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

This could be useful if the same left-hand-side is used for several right-hand-sides. The factorization can also be used for calculating the determinant because $\det(A)=\det(PLU)=\det(P)\det(U)=\pm \prod u_{ii}$ because $U$ is triangular and the sign is determined from $\det(P)$.

In [17]:
det(Alu)

-2.4921240072770106

# QR
When `A` is tall, Julia computes the least squares solution $\hat{x}$ that minimizes $\|Ax-b\|_2$. This can be done by factorizing $A=QR$ where $Q$ is unitary/orthogonal and $R=\left(\begin{smallmatrix}R_0\\0\end{smallmatrix}\right)$ and $R_0$ is upper triangular. With the QR factorization the minimum norm can be expressed
\begin{equation*}
\|Ax-b\|=\|QRx-b\|=\|Q(Rx-Q'b)\|=\|Rx-Q'b\|=\left\|\begin{pmatrix}R_0x-Q_0'b\\Q_1'b\end{pmatrix}\right\|=\|R_0x-Q_0'b\|+\|Q_1'b\|
\end{equation*}
and the problem therefore reduces to solving the square problem $R_0x=Q_0'b$ for $x$.

We can QR factorize the submatrix of the two first columns of $A$ by

In [19]:
Aqr = qrfact(A[:,1:2])

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}(3x2 Array{Float64,2}:
 0.920302    0.88955 
 0.0499625  -1.90453 
 0.412267   -0.105392,2x2 Array{Float64,2}:
 1.70582       -0.0219749
 2.15132e-314   1.97803  )

`\` has a method for the QR and the least squares problem is therefore solved with

In [20]:
Aqr\b

2-element Array{Float64,1}:
 1.55699 
 0.935888

It should be noted that this is *not* the way `A[:,1:2]\b` is solved. In order to handle rank deficient problems Julia uses a QR factorization with pivoting. Pivoting is enabled with a special boolean argument

In [21]:
Aqrp = qrfact([A[:,1] A[:,1]], Val{true})

Base.LinAlg.QRPivoted{Float64,Array{Float64,2}}(3x2 Array{Float64,2}:
 0.920302   0.920302   
 0.0499625  1.11886e-16
 0.412267   0.882782   ,[1.7058151351483488,1.1240347345892086],[1,2])

Notice that the type is different now. `\` also has a method for `QRPivoted` and the rank deficient problem is therefore computed

In [22]:
Aqrp\b

2-element Array{Float64,1}:
 1.2308
 1.2308

Another feature of the QR factorizations is the `Q` types for storing the unitary matrices $Q$. They can be extracted from the different from `QR` types by indexing

In [23]:
Aqr[:Q]

3x3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.705815   -0.0632519  -0.705567
 -0.0852268  -0.981189    0.173217
 -0.703251    0.182393    0.687147

The matrix has a compact internal representation

In [24]:
@show fieldnames(Aqr[:Q])
V = tril(Aqr[:Q].factors, -1)
V[1,1] = V[2,2] = 1
@show V
@show T = Aqr[:Q].T;

# Hence the complete reflector can be computed 
I - V*T*V'

fieldnames(Aqr[:Q]) = [:factors,:T]

3x3 Array{Float64,2}:
 -0.705815   -0.0632519  -0.705567
 -0.0852268  -0.981189    0.173217
 -0.703251    0.182393    0.687147

Even though the `Q` type is printed as a square matrix, it is in practice representing the  $3\times 2$ version as well. Hence both

In [25]:
Aqr[:Q]*ones(2)

3-element Array{Float64,1}:
 -0.769067
 -1.06642 
 -0.520858

and

In [26]:
Aqr[:Q]*ones(3)

3-element Array{Float64,1}:
 -1.47463 
 -0.893199
  0.166289

works, but not

In [27]:
Aqr[:Q]*ones(4)

LoadError: LoadError: DimensionMismatch("vector must have length either 3 or 2")
while loading In[27], in expression starting on line 1

#### Function overloading/mutiple dispatch

In [18]:
@which A_ldiv_B!(Alu, b)

A_ldiv_B!{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S<:Union{DenseArray{T,2},SubArray{T,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}}(A::Base.LinAlg.LU{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S<:Union{DenseArray{T,2},SubArray{T,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}}, B::Union{DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},1},DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},1,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/lu.jl:133

In [29]:
@which A_ldiv_B!(Aqr, b)

A_ldiv_B!{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64}}(A::Base.LinAlg.QRCompactWY{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},M<:AbstractArray{T,2}}, b::Union{DenseArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},1},SubArray{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},1,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/qr.jl:299

### Eigendecompositions and the SVD(s)

The results from eigendecompositions and singular values decompositions are also stored in `Factorization` types. This also include Hessenberg and Schur factorizations.

The eigendecomposition can be computed

In [30]:
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-2.197030178391658,-0.6475253632158462,3.907743114832829],3x3 Array{Float64,2}:
 0.663497  -0.743686    0.0818729
 0.217108   0.0866579  -0.972293 
 0.715986   0.662889    0.218958 )

The values and the vectors can be extracted from the Eigen type by special indexing

In [31]:
AsymEig[:values]

3-element Array{Float64,1}:
 -2.19703 
 -0.647525
  3.90774 

In [32]:
AsymEig[:vectors]

3x3 Array{Float64,2}:
 0.663497  -0.743686    0.0818729
 0.217108   0.0866579  -0.972293 
 0.715986   0.662889    0.218958 

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^T)^{-1}=V\Lambda^{-1}V^T$.

In [33]:
inv(AsymEig)*Asym

3x3 Array{Float64,2}:
 1.0          -1.88738e-15  -2.33147e-15
 3.05311e-16   1.0           9.99201e-16
 0.0           1.55431e-15   1.0        

Julia also has an `eig` function which returns a tuple with the values and the vectors

In [34]:
eig(Asym)

([-2.197030178391658,-0.6475253632158462,3.907743114832829],
3x3 Array{Float64,2}:
 0.663497  -0.743686    0.0818729
 0.217108   0.0866579  -0.972293 
 0.715986   0.662889    0.218958 )

This is mainly provided for MATLAB compatibility.

The `svdfact` function computes the singular value decomposition

In [35]:
Asvd = svdfact(A[:,1:2])

Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}(3x2 Array{Float64,2}:
  0.29581   0.64395 
 -0.809151  0.561489
  0.507712  0.51967 ,[2.144116717870728,0.8174661132840748],2x2 Array{Float64,2}:
 -0.213269  -0.976993
 -0.976993   0.213269)

and again `\` has a method for the type enabling least squares by SVD

In [36]:
Asvd\b

2-element Array{Float64,1}:
 1.55699 
 0.935888

In contrast to MATLAB, Julia does not allow dispatch on the number of output arguments and therefore there are special functions for providing values only: `eigvals` and `svdvals`.

### Sparse factorizations

In [37]:
AS = sprandn(1000, 1000, 0.01) + I
bb = randn(1000)
@show lufact(AS)
@show cholfact(AS'AS)
@show ldltfact(AS + AS')
@show qrfact(AS[:,1:10])
;

lufact(AS) = UMFPACK LU Factorization of a 1000-by-1000 sparse matrix
Ptr{Void} @0x00007fd0907d7dc0

cholfact(AS' * AS) = Base.SparseMatrix.CHOLMOD.Factor{Float64}
type:          LLt
method: supernodal
maxnnz:          0
nnz:        460823

ldltfact(AS + AS') = Base.SparseMatrix.CHOLMOD.Factor{Float64}
type:         LDLt
method: simplicial
maxnnz:     251798
nnz:        251798

qrfact(AS[:,1:10]) = Base.SparseMatrix.SPQR.Factorization{Float64}(1000,10,Ptr{Base.SparseMatrix.SPQR.C_Factorization{Float64}} @0x00007fd090a3a210)

In [38]:
# lufact based
(AS\bb)[1:3]

3-element Array{Float64,1}:
 -15.433 
 -18.2876
  12.4687

In [39]:
# cholfact based
((AS'AS)\(AS'bb))[1:3]

3-element Array{Float64,1}:
 -15.433 
 -18.2876
  12.4687

In [40]:
# ldltfact based
((AS + AS')\bb)[1:3]

3-element Array{Float64,1}:
  -3.02018
 -16.7284 
 -16.1353 

In [41]:
# qrfact based
AS[:,1:3]\bb

3-element Array{Float64,1}:
 -0.227766 
 -0.460589 
 -0.0407802

##Special matrices
The structure of matrices is very important in linear algebra. This structure can be made explicit in Julia through composite types. Examples are `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`. Specialized methods are written for the special matrix types to take advantage of their structure. Below some examples are shown

In [42]:
Diagonal(diag(A))

3x3 Diagonal{Float64}:
 -0.649563  0.0       0.0     
  0.0       1.79289   0.0     
  0.0       0.0      -0.611733

In [43]:
LowerTriangular(A)

3x3 LowerTriangular{Float64,Array{Float64,2}}:
 -0.649563    0.0        0.0     
 -0.0784344   1.79289    0.0     
 -0.647203   -0.972948  -0.611733

In [44]:
Symmetric(Asym)

3x3 Symmetric{Float64,Array{Float64,2}}:
 -1.29913   -0.585827  -0.654438
 -0.585827   3.58578   -1.21064 
 -0.654438  -1.21064   -1.22347 

In [45]:
SymTridiagonal(diag(Asym), diag(Asym,1))

3x3 SymTridiagonal{Float64}:
 -1.29913   -0.585827   0.0    
 -0.585827   3.58578   -1.21064
  0.0       -1.21064   -1.22347

When it is known that a matrix is e.g. triangular or symmetric Julia might be able to solve a problem faster by converting the matrix to a special matrix. For some of the procedures, Julia checks if the input matrix is triangular or symmetric and converts the matrix if such a structure is detected. It should be noted that `Symmetric`, `Hermitian` and `Triangular` do not copy the input matrix.
### Symmetric eigenproblem
Whether or not Julia is able to detect if a matrix is symmetric/Hermitian can have a big influence on how fast an eigenvalue problem is solved. Sometimes it is known that a matrix is symmetric or Hermitian but due to floating point errors this is not detected at runtime by the `eigvals` function. In the following example `Asym1` and `Asym2` are almost identical, but unless Julia is told that `Asym2` is symmetric, the elapsed time for the computation is very different.

In [46]:
n = 1000;
A = randn(n, n);
Asym1 = A + A';
Asym2 = copy(Asym1);Asym2[1,2] += 5eps();
println("Is Asym1 symmetric? ", issym(Asym1))
println("Is Asym2 symmetric? ", issym(Asym2))

Is Asym1 symmetric? true

In [47]:
@time eigvals(Asym1);


Is Asym2 symmetric? false
  

In [48]:
@time eigvals(Asym2);

0.379205 seconds (506 allocations: 8.018 MB)
  

In [49]:
@time eigvals(Symmetric(Asym2));

0.943954 seconds (45 allocations: 8.177 MB, 0.42% gc time)
  

In [50]:
@which eigvals(Asym2)

eigvals{T}(A::Union{DenseArray{T,2},SubArray{T,2,A<:DenseArray{T,N},I<:Tuple{Vararg{Union{Colon,Int64,Range{Int64}}}},LD}}) at linalg/eigen.jl:91

In [51]:
@which eigvals(Symmetric(Asym2))

eigvals{T1<:Real,T2}(A::Union{Hermitian{Complex{T1<:Real},T2},Hermitian{T1<:Real,T2},Symmetric{T1<:Real,T2}}) at linalg/symmetric.jl:144

We are considering a less restrictive test for symmetry that allows small floating point errors in symmetric matrices. However, explicit type declaration still avoids the runtime check.

### A big problem
By using the tridiagonal matrices it is possible to work with potentially very large problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a `Matrix` type.

In [52]:
using Distributions
n = 1_000_000;
A = SymTridiagonal(2randn(n), Float64[rand(Chi(i)) for i = n-1:-1:1]);
@time eigmin(A), eigmax(A) # Extremal eigenvalues of the Hermite ensemble

INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/Distributions.ji for module Distributions.


0.195390 seconds (6.46 k allocations: 8.267 MB)


INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/ArrayViews.ji for module ArrayViews.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/PDMats.ji for module PDMats.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/StatsFuns.ji for module StatsFuns.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/StatsBase.ji for module StatsBase.


(-1999.9421881709475,2000.1887073730302)

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does. For a long time Julia has also had support for multiplicaton of general element types. Hence, when multiplying integer matrices, the output is also an integer matrix

In [53]:
rand(1:10,3,3)*rand(1:10,3,3)

3x3 Array{Int64,2}:
 27  16  12
 35  23  17
 48  33  25

1.425342 seconds (225.98 k allocations: 422.985 MB, 7.07% gc time)


Recently, more generic linear algebra methods were added and Julia now supports generic `LU` and `QR` factorizations. Generic eigenvalue methods are being tested and can hopefully be added to Julia soon.

In general, the `LU` factorization can be computed whenever the matrix element type is closed under the operations `+`, `-`, `*` and `\`. Of couse the matrix also has to have full rank. The generic `LU` method in Julia applies pivoting and therefore the element type also has to support `<` and `abs`. In consequence it is possible to solve systems of equations of e.g. rational numbers which the following examples show.

This functionality is primarily useful in teaching or exploratory work as floating point values are dominating in real applications. However, arbitrary precision and efficient integer arithmetic have real world applications and are not supported in LAPACK.

### Example 1: Rational linear system of equations
Julia has rational numbers built in. The following example shows how a linear system of equations can be solved without promoting to floating point element types. Overflow can easisly become a problem when working with rational numbers so we use `BigInt`s.

In [54]:
Ar = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10
@show Ar
x = ones(Int,3)
b = Ar*x
Ar\b

Ar = Rational{BigInt}[4//5 3//10 3//10
                 3//10 3//5 3//5
                 1//5 9//10 9//10]

LoadError: LoadError: Base.LinAlg.SingularException(3)
while loading In[54], in expression starting on line 5

### Example 2: Rational matrix from eigenstructure
The next example shows how rational matrix arithmetic can be used for calculating a matrix given rational eigenvalues and -vectors. I find this convenient when giving examples of linear dynamic systems.

In [55]:
λ1, λ2, λ3 = 1//1, 1//2, 1//4
v1, v2, v3 = [1,0,0], [1,1,0], [1,1,1]
V, Λ = [v1 v2 v3], Diagonal([λ1, λ2, λ3])
@show V
@show Λ
A = V*Λ/V


V = [1 1 1
 0 1 1
 0 0 1]
Λ = Rational{Int64}[1//1 0//1 0//1
                0//1 1//2 0//1
                0//1 0//1 1//4]

3x3 Array{Rational{Int64},2}:
 1//1  -1//2  -1//4
 0//1   1//2  -1//4
 0//1   0//1   1//4

### Example 3: The Hilbert matrix.
The Hilbert matrix $H$ defined by $(h_{ij})=\frac{1}{i+j-1}$ is very ill conditioned. Even for small dimensions, the inverse of the Hilbert matrix is very imprecise in double precision. By defining the matrix with rational elements, the inverse can be calculated exact

In [56]:
nHilbert = 8
H = Rational{BigInt}[1//(i+j-1) for i = 1:nHilbert,j = 1:nHilbert]

8x8 Array{Rational{BigInt},2}:
 1//1  1//2  1//3   1//4   1//5   1//6   1//7   1//8 
 1//2  1//3  1//4   1//5   1//6   1//7   1//8   1//9 
 1//3  1//4  1//5   1//6   1//7   1//8   1//9   1//10
 1//4  1//5  1//6   1//7   1//8   1//9   1//10  1//11
 1//5  1//6  1//7   1//8   1//9   1//10  1//11  1//12
 1//6  1//7  1//8   1//9   1//10  1//11  1//12  1//13
 1//7  1//8  1//9   1//10  1//11  1//12  1//13  1//14
 1//8  1//9  1//10  1//11  1//12  1//13  1//14  1//15

In [57]:
inv(H)

8x8 Array{Rational{BigInt},2}:
      64//1      -2016//1       20160//1  …       192192//1      -51480//1
   -2016//1      84672//1     -952560//1       -10594584//1     2882880//1
   20160//1    -952560//1    11430720//1       141261120//1   -38918880//1
  -92400//1    4656960//1   -58212000//1      -776936160//1   216216000//1
  221760//1  -11642400//1   149688000//1      2118916800//1  -594594000//1
 -288288//1   15567552//1  -204324120//1  …  -3030051024//1   856215360//1
  192192//1  -10594584//1   141261120//1      2175421248//1  -618377760//1
  -51480//1    2882880//1   -38918880//1      -618377760//1   176679360//1

Even for $n=8$ the difference between the exact and the floating point result is big.

In [58]:
norm(map(Float64, inv(H)) - inv(map(Float64, H)), Inf)

439.45201206207275

Another approach could be to use higher precision floating point numbers.
Higher precision floating point numbers are built into Julia and it is therefore easy to convert the Hilbert matrix

In [59]:
Hbf = convert(Matrix{BigFloat}, H);
Float64(norm(inv(Hbf) - inv(H), Inf))

6.304476724749898e-59

Which is fairly precise. However, the matrix need not grow much before 256 bit floats are not sufficient

In [60]:
nHilbert = 30
H = Rational{BigInt}[1//(i+j-1) for i = 1:nHilbert,j = 1:nHilbert];
Hbf = convert(Matrix{BigFloat}, H);
Float64(norm(inv(Hbf) - inv(H), Inf))

2.1751007413310495e8

The next example shows how the LU factorization can be used for a user defined number type.
### Example 4: GF($p$)
Julia makes it easy to define new number types. The block below defines a type representing the Galois field with $p$ elements GF($p$). Please note how few lines it takes!

In [61]:
# Scalar finite fields
immutable GF{P,T<:Integer} <: Number
    data::T
    function GF(x::Integer)
        return new(mod(x, P))
    end
end

In [62]:
# methods for scalar finite field
import Base: convert, inv, one, promote_rule, show, zero

function call{P}(::Type{GF{P}}, x::Integer)
    if !isprime(P)
        throw(ArgumentError("P must be a prime"))
    end
    return GF{P,typeof(x)}(mod(x, P))
end
convert{P,T}(::Type{GF{P,T}}, x::Integer) = GF{P}(x)
convert{P}(::Type{GF{P}}, x::Integer) = GF{P}(x)
convert{P,T}(::Type{GF{P,T}}, x::GF{P}) = GF{P,T}(x.data)
promote_rule{P,T1,T2<:Integer}(::Type{GF{P,T1}}, ::Type{T2}) = GF{P,promote_type(T1,T2
)}
show(io::IO, x::GF) = show(io, x.data)

show (generic function with 120 methods)

In [63]:
GF{3}(11)

2

define basic arithmetic with meta programming

In [64]:
import Base: +, -, *

for op in (:+,:-,:*)
    @eval begin
        ($op){P,T}(x::GF{P,T}, y::GF{P,T}) = GF{P,T}($(op)(x.data, y.data))
    end
end

In [65]:
x, y = GF{5}(4), GF{5}(3)
@show x + y
@show x - y
@show x * y
;

x + y = 2

In [66]:
# Division requires slightly more care
import Base: /

function inv{P,T}(x::GF{P,T})
    if x == zero(x)
        throw(DivideError())
    end
    r, u, v = gcdx(x.data, P)
    GF{P,T}(u)
end
(/){P}(x::GF{P}, y::GF{P}) = x*inv(y)
;

In [67]:
@show x / y
@show x \ y # we get this by inheritance from Number;

 We can now create a square matrix and a vector over $GF(5)$

In [68]:
srand(1234)
A = [GF{5}(rand(0:4)) for i = 1:4, j = 1:4]

4x4 Array{GF{5,Int64},2}:
 3  1  1  0
 1  3  4  3
 4  3  2  2
 0  0  1  1

In [69]:
b = [GF{5}(rand(0:4)) for i = 1:4]


x - y = 1
x * y = 2
x / y = 3
x \ y = 2


4-element Array{GF{5,Int64},1}:
 4
 1
 2
 0

Because the LU factorization in Julia works whenever the matrix elements form a field

In [70]:
x = A\b

4-element Array{GF{5,Int64},1}:
 2
 3
 0
 0

In [71]:
A*x - b

4-element Array{GF{5,Int64},1}:
 0
 0
 0
 0

It worked! Our LU algorithm automatically turns off pivoting when the elements are not floating point types. This is necessary here because finite fields cannot be ordered and therefore partial pivoting is not defined. In consequence, the LU factorization might not be defined for some matrices even when they have full rank because of zero pivots

In [72]:
A[1,1] = 0 # works evens though 0 is an integer because of the promotion rule defined above
lufact(A)  # Indeed, the error message could be improved here

LoadError: LoadError: MethodError: `real` has no method matching real(::GF{5,Int64})
while loading In[72], in expression starting on line 2

Eventually, we'd like to pivot on the first non-zero elements in the column which would ensure existence of the LU for all full rank matrices.

What are the costs of user-defined types? Usually, there is a large difference between a user-defined type and built-in type. With some care, this dichotomy between built-in and user-defined types is not existing. Compare

In [73]:
A1, A2 = rand(1:100, 100, 100), rand(1:100, 100, 100)
A1*A2 # warm up to be sure function is compiled
@time A1*A2
AF1, AF2 = map(GF{5}, A1), map(GF{5}, A2)
AF1*AF2
@time AF1*AF2
;

  0.000760 seconds (13 allocations: 78.656 KB)
  

In [74]:
@time lufact(A1) # Promoted to Float64 and calls LAPACK
F = lufact(AF1,Val{false})
while F.info != 0
    AF1[F.info, F.info] += 1
    F = lufact(AF1, Val{false})
end
@time lufact(AF1) # Non-blocked generic LU implemented in Julia
;

0.006990 seconds (13 allocations: 78.656 KB)
  0.000390 seconds (8 allocations: 79.250 KB)
  

Costs are mainly related to the modulo calculation. Optimized methods for $GF(2)$ could be added easily.

When a type has been defined as an `immutable` and if the fields are all pointer free, as in this example, it is possible to have memory locality for custom types. This can be exemplified with the use of pointers where we just walk linearly through the memory

In [75]:
@show b
@show pb = pointer(b)
for i = 1:length(b)
    @show unsafe_load(pb, i)
end

### Example 5: Quaternions
The final example of generic linear algebra in Julia shows how the generic elementary reflectors work. The first step in solving a Hermitian eigenvalue problem is to tranform the Hermitian matrix to a real symmetric tridiagonal matrix by elementary reflectors of the form 

$$H=I-v\tau v'$$

satisfying

$$H'H=I,$$ 

i.e. they are unitary. Most often, textbooks present the special case of Householder reflectors $H=I-2vv'$ where $v'v=1$. Householder reflectors have the nice property that they are also Hermitian, but they fail to reduce Hermitian matrix to a *real symmetric* tridiagonal matrix. The following example shows how the underlying real symmetric tridiagonal eigenproblem can be extracted from a Hermitian eigenvalue problem of a matrix with a user-defined element type.

In my github repo, I have a [package](https://github.com/andreasnoack/LinearAlgebra.jl.git) that defines generic eigen and singular value decompositions and some more. Recall that quaternions generalize complex numbers and can be written $q=x_1+x_2i+x_3j+x_4k$ where the "imaginary" units $i$,$j$ and $k$ satisfy $i^2=j^2=k^2=ijk=-1$. The only difficulty in defining quaternions is the fact that they do not commute, e.g.
$$
ij=-ijkk=k
$$
but
$$
ji=-jiijk=jjk=-k.
$$ However, the generic algorithms for LU factorization and elementary reflectors do not assume commutativity and therefore we can solve positive definite eigenproblems for quaternions with the exact same function as we use in the real case. First we compute a Hermitian matrix of quaternions

In [76]:
using Quaternions
using LinearAlgebra
import Base: convert, isreal
function convert{T<:FloatingPoint}(::Type{FloatingPoint}, q::Quaternion{T})
    if !isreal(q)
        throw(InexactError())
    end
    convert(T, q.s)
end

isreal(q::Quaternion) = q.v1 == 0 && q.v2 == 0 && q.v3 ==0

A = Quaternion{Float64}[Quaternion(randn(4)...) for i = 1:2,j=1:2]
Aher = A + A' # Warning! Matrices of quaternions don't print nicely

INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/DualNumbers.ji for module DualNumbers.


0.003355 seconds (8 allocations: 79.250 KB)
b = GF{5,Int64}[4,1,2,0]
pb = pointer(b) = Ptr{GF{5,Int64}} @0x0000000109611000
unsafe_load(pb,i) = 4
unsafe_load(pb,i) = 1
unsafe_load(pb,i) = 2
unsafe_load(pb,i) = 0


INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/NaNMath.ji for module NaNMath.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/Calculus.ji for module Calculus.
  likely near In[76]:4
  likely near In[76]:4


2x2 Array{Quaternions.Quaternion{Float64},2}:
                                              -2.4517927171581895 + 0.0im + 0.0jm + 0.0km  …  0.018837251573537772 + 3.4747377789843217im + 1.1782923757360937jm - 1.265617102269836km
 0.018837251573537772 - 3.4747377789843217im - 1.1782923757360937jm + 1.265617102269836km                                                 -0.33813735803339834 + 0.0im + 0.0jm + 0.0km

In [77]:
fieldnames(Aher[1])

5-element Array{Symbol,1}:
 :s   
 :v1  
 :v2  
 :v3  
 :norm

We can now calculate the eigenvalues. In this case, we need to call the Hermitian constructor explicitly because Julia Base only checks if the matrix is Hermitian when the element type is supported by LAPACK.

In [78]:
# reload("LinearAlgebra")
LinearAlgebra.EigenSelfAdjoint.eigvals!(Hermitian(copy(Aher), :L))

2-element Array{Float64,1}:
 -5.41755
  2.62762

We can break down the steps of the calculation. First, the matrix is transformed to a symmetric tridiagonal matrix

In [79]:
AherFact = LinearAlgebra.EigenSelfAdjoint.symtri!(Hermitian(copy(Aher), :L))
typeof(AherFact)

LinearAlgebra.EigenSelfAdjoint.SymmetricTridiagonalFactorization{Quaternions.Quaternion{Float64}}

The type does not print nicely and it is slightly more informative to get the property names

In [80]:
fieldnames(AherFact)

3-element Array{Symbol,1}:
 :factors      
 :scalarfactors
 :diagonals    

The property `factors` is a matrix with quaternion elements whose diagonal and subdiagonal contain the real symmetric tridiagonal matrix, i.e. the $i$,$j$ and $k$ terms are all zero. The elements below the subdiagonal contain the elements of the vectors $v$ of the elementary reflectors. The property `scalafactors` contains the multiplyers $\tau$. Finally, the property `diagonals` contains a real  `SymTridiagonal` matrix. 

When we do not assume commutativity, an elementary reflector is written $H=I-v\tau v^*$. As in `LAPACK`, we use the convention that the first element of $v$ is one. Hence, we can write the factorization $A + A^*=QSQ^*$ where $Q$ is unitary and $S$ is a tridiagonal matrix.

In [81]:
Q = [1 0;0 1 - AherFact.scalarfactors[1]];
S = full(AherFact.diagonals);
norm(Aher-Q*S*Q',1)

4.770756440442562e-16

All arithmetic operations are more expensive because a quaternion contains four real elements. Furthermore, LAPACK is fast because it uses blocked algorithms for the reduction to symmetric tridiagonal form which we do not in our generic code. Anyway, the reduction to a real symmetric tridiagonal matrix can be done fairly fast. Below we present some results for computational time and memory usage.
#### Real

In [82]:
n = 500;
A = randn(n,n);
Apd = A'A;
@time eigvals(Symmetric(Apd));

#### Complex

In [83]:
A = complex(randn(n,n),randn(n,n));
Apd = A'A;
@time eigvals(Hermitian(Apd));

0.046756 seconds (29 allocations: 2.088 MB)
  

#### Quaternion


In [84]:
A = Quaternion{Float64}[Quaternion(randn(4)...) for i = 1:n,j=1:n];
Apd = A'A;
for i = 1:n
    Apd[i,i] = real(Apd[i,i]) # The generic matrix multiplaction makes tiny error in imaginary parts
end
@time LinearAlgebra.EigenSelfAdjoint.eigvals!(Hermitian(copy(Apd), :L));

0.188826 seconds (200.76 k allocations: 12.686 MB, 2.63% gc time)
  

Notice that only the real and complex versions benefit from optimized BLAS-3 kernels.

### Example 6: Symbolic linear algebra with SymPy+Julia.
Julia is able to call Python mainly due to Steven G. Johnson. The linear algebra in Julia is also generic for elements of SymPy values wrapped through Julia's PyCall functionality.

Below is an example of symbolic arrow matrix

In [85]:
using SymPy
z = Sym[Sym("zeta_$i") for i = 1:3]
D = diagm(Sym["delta_$i" for i = 1:3])
A = [Sym("alpha") z'; z D]

INFO: Precompiling module SymPy...


0.951917 seconds (520 allocations: 9.652 MB, 0.24% gc time)


INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/PyCall.ji for module PyCall.
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:44
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:44
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:49
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:49
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:52
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/getthing.jl:53
  likely near /Users/andreasnoack/.julia/v0.4/Requires/src/require.jl:32
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/Reexport.ji for module Reexport.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/Colors.ji for module Colors.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/FixedPointNumbers.ji for module FixedPointNumbers.
INFO: Recompiling stale cache file /Users/andreasnoack/.j

[Plots.jl] Default backend: 

Expr(:import, :DataFrames)::Any
  ** incremental compilation may be broken for this module **



...

INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/DataFrames.ji for module DataFrames.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/DataArrays.ji for module DataArrays.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/GZip.ji for module GZip.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/SortingAlgorithms.ji for module SortingAlgorithms.
INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/Docile.ji for module Docile.
Expr(:import, :DataFrames)::Any
  ** incremental compilation may be broken for this module **

Expr(:function, Expr(:call, :createKWargsList, Expr(:parameters, Expr(:..., :kw)::Any)::Any, Expr(:::, :plt, :PlottingObject)::Any, Expr(:::, :df, Expr(:., :DataFrames, :DataFrame)::Any)::Any, Expr(:..., :args)::Any)::Any, Expr(:block, # line 366 /Users/andreasnoack/.julia/v0.4/Plots/src/plot.jl, Expr(:call, :createKWargsList, Expr(:parameters, Expr(:..., :kw)::Any, Ex

[Plots.jl] Default backend: pyplot

INFO: Installing sympy via the Conda package...



Fetching package metadata: ....
Solving package specifications: .................
Package plan for installation in environment /Users/andreasnoack/.julia/v0.4/Conda/deps/usr:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fastcache-1.0.2            |           py27_0          23 KB
    sympy-0.7.6.1              |           py27_0         6.2 MB
    ------------------------------------------------------------
                                           Total:         6.2 MB

The following NEW packages will be INSTALLED:

    fastcache: 1.0.2-py27_0  
    sympy:     0.7.6.1-py27_0

Fetching packages ...
fastcache-1.0. 100% |###############################| Time: 0:00:00 547.99 kB/s
sympy-0.7.6.1- 100% |###############################| Time: 0:00:02   2.61 MB/s
Extracting packages ...
[      COMPLETE      ]|##################################################| 100%
Linking packages ...
[  

4x4 Array{SymPy.Sym,2}
[        ______   ______   ______ ]
[alpha   zeta_1   zeta_2   zeta_3 ]
[                                 ]
[zeta_1  delta_1     0        0   ]
[                                 ]
[zeta_2     0     delta_2     0   ]
[                                 ]
[zeta_3     0        0     delta_3]

In [86]:
simplify(det(A))

                                                       ______                 
alpha*delta_1*delta_2*delta_3 - delta_1*delta_2*zeta_3*zeta_3 - delta_1*delta_

         ______                          ______
3*zeta_2*zeta_2 - delta_2*delta_3*zeta_1*zeta_1

In [87]:
A[4,4] = 0
map(simplify, inv(A))

4x4 Array{SymPy.Sym,2}
[                                                                          1  
[  0           0               0                                         -----
[                                                                        zeta_
[                                                                             
[              1                                                        -zeta_
[  0        -------            0                                     ---------
[           delta_1                                                  delta_1*z
[                                                                             
[                              1                                        -zeta_
[  0           0            -------                                  ---------
[                           delta_2                                  delta_2*z
[                                                                             
[            ______          

The arithmetic operations are overloaded with calls to SymPy and therefore Julia's generic linear algebra routines work.

###Example 7: Promotion of elements
When calculating factorizations, promotion of elements of matrices and vectors is defined through the promotion rules of the elements for the operations necessary for the factorization. In this way, it is not necessary to know about all possible element types for which the factorization could possibly be defined as long as promotion rules are defined for the element types themselves. Below is a simplified version of the promotion used in the LU factorization.

In [88]:
# simplified (no pivoting option) so code doesn't run
function lufact{T}(A::AbstractMatrix{T})
    S = typeof(zero(T)/one(T))
    lufact!(copy_oftype(A, S))
end

LoadError: LoadError: error in method definition: function LinAlg.lufact must be explicitly imported to be extended
while loading In[88], in expression starting on line 2

##Julia is fast
Most high level languages have substantial parts of the language written in C for performance reasons. The majority of Julia's standard library is written in Julia itself.

This is possible without a performance penalty bacuase, via LLVM, Julia code compiles into efficient machine code when possible even without type annotations

In [89]:
@code_native 1 + 1

	.section	__TEXT,__text,regular,pure_instructions
Filename: int.jl
Source line: 8
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 8
	addq	%rsi, %rdi
	movq	%rdi, %rax
	popq	%rbp
	ret


It is important for *performance* that input type maps to output type unambiguously. In this case it is possible to statically analyze the program, i.e. the types can be checked within subprograms. When the return type is ambiguous, Julia relies on a dynamic lookup of the right method based on the actual runtime value.

In [90]:
isone1(x) = x == 1 ? true : 0      # not type stable. Output _type_ depends on input _value_
isone2(x) = x == 1 ? true : false;

In [91]:
@time for i = 1:10^6; isone1(rand() > 0.5); end
@time for i = 1:10^6; isone2(rand() > 0.5); end

Mixing statically and dynamic methods invisibly is convenient for prototyping such that the program doesn't fail when the types cannot be checked statically. Julia provides helper functions for detecting code that cannot be checked statically.

In [92]:
@code_warntype isone1(1)

  0.008982 seconds
  0.002775 seconds
Variables:
  x::Int64

Body:
  begin 

In [93]:
@code_warntype isone2(1)

In [94]:
@code_llvm isone1(1) # note the $jl_true in the assembly

In [95]:
@code_llvm isone2(1)

### Julia for numerical libraries?

#### LAPACK?
Julia can be (almost) as fast as well-written Fortran. I.e. the abstractions have over small overhead compared to working with raw pointers.

Below, I'll give some LAPACK vs Julia comparisons. The Julia implementations are taken from my experimental [LinearAlgebra](https://github.com/andreasnoack/LinearAlgebra) repository.

##### Blocked Cholesky
$$
\begin{pmatrix}
A_{11} & A_{12} \\
A_{21} & A_{22} \\
\end{pmatrix} = 
\begin{pmatrix}
L_{11} & 0 \\
L_{21} & L_{22}
\end{pmatrix}
\begin{pmatrix}
L_{11}^* & L_{21}^* \\
0 & L_{22}^*
\end{pmatrix}
$$
so blocked algorithm consists of
- Cholesky factorize: $A_{11}=L_{11}L_{11}^*$
- Triangular solve: $L_{21} = A_{21}L_{11}^{-1*}$
- Symmetric update: $A_{22} := A_{22} - L_{21}L_{21}^*$

In [96]:
using LinearAlgebra

# Cholesky
n = 2000
A = randn(n,n)
A = A'A

chol(A)
gc()
@time C_LAPACK = chol(A) # LAPACK

LinearAlgebra.CholeskyModule.cholRec!(copy(A), Val{:L}, 1)
gc()
@time C_JuliaRec = LinearAlgebra.CholeskyModule.cholRec!(copy(A), Val{:L}, 1)

LinearAlgebra.CholeskyModule.cholBlocked!(copy(A), Val{:L}, 40)
gc()
@time C_JuliaBlocked = LinearAlgebra.CholeskyModule.cholBlocked!(copy(A), Val{:L}, 40)
;

 # In[90], line 1:
      unless x::Int64 === 1::Bool goto 0
      return true
      0: 
      return 0
  end::UNION{BOOL,INT64}
Variables:
  x::Int64

Body:
  begin  # In[90], line 2:
      unless x::Int64 === 1::Bool goto 0
      return true
      0: 
      return false
  end::Bool

define %jl_value_t* @julia_isone1_24521(i64) {
top:
  %1 = icmp eq i64 %0, 1
  br i1 %1, label %if, label %L

if:                                               ; preds = %top
  %2 = load %jl_value_t** @jl_true, align 8
  ret %jl_value_t* %2

L:                                                ; preds = %top
  ret %jl_value_t* inttoptr (i64 4353597520 to %jl_value_t*)
}

define i1 @julia_isone2_24522(i64) {
top:
  %not. = icmp eq i64 %0, 1
  ret i1 %not.
}
  0.090249 seconds (10 allocations: 30.518 MB, 1.72% gc time)
  0.085160 seconds (14.00 k allocations: 31.128 MB, 2.32% gc time)
  

It is not clear why Julia is *slightly* faster than LAPACK here. Call overhead should be negligible. It might be because of an untuned block size parameter in `ilaenv.f`.

Julia could make it much easier to experiment with values like blocking size. Julia allows for easy compilation of versions for different fixed blocking sizes. LLVM might be able to optimize the code further if blocking size was fixed at compile time (we could compile different versions and use operator overloading), but this is not pursued further here.

In [97]:
using PyPlot
C_Julia2 = LinearAlgebra.CholeskyModule.cholBlocked!(copy(A), Val{:L}, 10)
tb = Int[]
tt = Float64[]
for b in 20:5:100
    for i = 1:5
        push!(tb, b)
        gc()
        t = @elapsed LinearAlgebra.CholeskyModule.cholBlocked!(copy(A), Val{:L}, b)
        push!(tt, t)
    end
end
plot(tb, tt, "o")

INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/PyPlot.ji for module PyPlot.


0.111333 seconds (6.10 k allocations: 30.859 MB, 1.46% gc time)


INFO: Recompiling stale cache file /Users/andreasnoack/.julia/lib/v0.4/LaTeXStrings.ji for module LaTeXStrings.
INFO: Loading additional PyPlot commands for graphing for SymPy objects:
contour, quiver, plot_surface, plot_parametric_surface, and plot_implicit.
See ?sympy_plotting for some more details


LoadError: LoadError: UndefVarError: plot not defined
while loading In[97], in expression starting on line 13

##### Symmetric tridiagonal eigenproblem
When only values are requested, LAPACK uses a square root free QR algorithm described in Parlett's "The Symmetric Eigenvalue Problem". No BLAS-3 optimization is possible for this algorithm.

In [98]:
n = 400
A = SymTridiagonal(2randn(n), Float64[rand(Chi(i)) for i = n-1:-1:1]) # Hermite ensemple

# LAPACK:
eigvals(A)
@time vals_LAPACK = eigvals(A)

# Pure Julia
LinearAlgebra.EigenSelfAdjoint.eigvals(A)
@time vals_Julia = LinearAlgebra.EigenSelfAdjoint.eigvals(A)
@time vals_BigJulia = LinearAlgebra.EigenSelfAdjoint.eigvals!(big(A), parse(BigFloat, "1e-500")) # use smaller n for this option.

@show Float64(norm((vals_LAPACK .- vals_BigJulia)./vals_BigJulia))
@show Float64(norm((vals_Julia .- vals_BigJulia)./vals_BigJulia))
;

  0.004292 seconds (13 allocations: 13.156 KB)
  0.005173 seconds (8 allocations: 6.641 KB)
  2.291242 seconds (16.85 M allocations: 559.928 MB, 24.21% gc time)
Float64(norm((vals_LAPACK .- vals_BigJulia) ./ vals_BigJulia)) = 1.6309789298379056e-13
Float64(norm((vals_Julia .- vals_BigJulia) ./ vals_BigJulia)) = 2.8737057330577663e-13


#### BLAS?
Not right now, but more recent version of LLVM supports inline assembly. Julia can already leverage this and combined with Julia's metaprogramming it might be possible to generate efficient kernels for multiplication or other operations like FFTs.

### Other approaches
- Cython
- Numba
- PyPy

## The future
- Julia 0.4
 - Faster GC
 - Package precompilation
 - New documentation system
 - Lots of other stuff...
- Julia >0.4
 - Multithreading
 - Better distributed performance
 - Debugger?
 
 >...So, if you are also a greedy, unreasonable, demanding programmer, we want you to give it a try.

from the first blog post.